In [5]:
#@title хитмап из кругов
import json
import folium
from collections import defaultdict

find_word = 'взламывают'

# Пути к файлам
json_path = '/content/drive/MyDrive/Г/predict/crimes_spb.json'
html_output = '/content/drive/MyDrive/Г/predict/'+ find_word +'_heatmap.html'

# Загрузка данных
print("Чтение JSON-файла...")
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Фильтрация: ищем "стрельба" в полях 'Причина' или 'Причина АПК БГ'
print(f"Фильтрация и подсчёт частоты {find_word}...")
shooting_locations = defaultdict(int)

for record in data:
    # Получаем координаты
    lat = record.get('Широта')
    lon = record.get('Долгота')
    cause1 = record.get('Причина', '') or ''
    cause2 = record.get('Причина АПК БГ', '') or ''

    # Проверка наличия слова "стрельба"
    if find_word not in cause1.lower() and find_word not in cause2.lower():
        continue

    # Проверка координат
    if lat is None or lon is None:
        continue
    try:
        lat = float(lat)
        lon = float(lon)
    except (ValueError, TypeError):
        continue

    # Диапазон координат для СПб
    if not (59.0 <= lat <= 61.0) or not (29.0 <= lon <= 31.0):
        continue

    # Ключ — округлённые координаты (для группировки близких точек)
    key = (round(lat, 4), round(lon, 4))  # Точность ~10 метров
    shooting_locations[key] += 1

# Проверка: есть ли данные
if not shooting_locations:
    print(f"Не найдено ни одного события со словом {find_word}.")
else:
    print(f"Найдено {len(shooting_locations)} уникальных локаций со стрельбой.")

    # Создание карты
    map_center = [59.9343, 30.3351]  # Центр СПб
    m = folium.Map(location=map_center, zoom_start=11, tiles='OpenStreetMap')

    # Добавление меток
    max_count = max(shooting_locations.values())
    for (lat, lon), count in shooting_locations.items():
        # Определяем цвет в зависимости от частоты
        if count < 5:
            color = 'yellow'
        elif count < 10:
            color = 'orange'
        else:
            color = 'red'

        # Размер метки
        radius = 5 + count * 3  # минимум 5, максимум зависит от частоты

        folium.Circle(
            location=[lat, lon],
            radius=radius,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"<b>Координаты:</b> {lat}, {lon}<br><b>Количество инцидентов:</b> {count}",
            tooltip=f"{count} инцидентов"
        ).add_to(m)

    # Сохранение карты
    m.save(html_output)
    print(f"Карта сохранена: {html_output}")
    print(f"Откройте файл в браузере, чтобы увидеть 'горячие точки' {find_word}.")

# Чтение файла
with open(html_output, 'r', encoding='utf-8') as f:
    html_content = f.read()

# Проверим, не добавлен ли уже футер (защита от дублирования)
if "</body>" in html_content:
    # Вставляем перед </body>
    footer_html = f"""
    <div style="position: fixed; bottom: 0; left: 0; width: 100%; background-color: white; color: black;
                text-align: center; padding: 6px; font-family: Arial, sans-serif; font-size: 13px;
                font-weight: bold; z-index: 9999; border-top: 2px solid #aaa; box-shadow: 0 -2px 5px rgba(0,0,0,0.1);
                pointer-events: none; display: flex; justify-content: center; align-items: center; height: 30px;">
        <div style="pointer-events: auto;">
            Карта событий с тегом: <{find_word}> | На основании периода 27.07.2023 - 27.07.2025
        </div>
    </div>
    """
    html_content = html_content.replace("</body>", footer_html + "\n</body>")

    # Перезаписываем файл
    with open(html_output, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"Футер добавлен, файл перезаписан: {html_output}")
else:
    print("Ошибка: не удалось найти место для вставки футера в HTML.")

Чтение JSON-файла...
Фильтрация и подсчёт частоты взламывают...
Найдено 9742 уникальных локаций со стрельбой.
Карта сохранена: /content/drive/MyDrive/Г/predict/взламывают_heatmap.html
Откройте файл в браузере, чтобы увидеть 'горячие точки' взламывают.
Футер добавлен, файл перезаписан: /content/drive/MyDrive/Г/predict/взламывают_heatmap.html


In [ ]:
!pip install pandas openpyxl folium

In [ ]:
#@title хитмап
import json
import folium
from folium.plugins import HeatMap

# Пути к файлам
json_path = '/content/drive/MyDrive/Г/predict/crimes_spb.json'
html_output = '/content/drive/MyDrive/Г/predict/shooting_heatmap_v2.html'

# Загрузка данных
print("Чтение JSON-файла...")
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Список координат для тепловой карты
heat_data = []

print("Фильтрация событий со словом 'стрельба' и сбор координат...")
for record in data:
    # Проверка причины
    cause1 = record.get('Причина', '') or ''
    cause2 = record.get('Причина АПК БГ', '') or ''

    if 'стрельба' not in cause1.lower() and 'стрельба' not in cause2.lower():
        continue

    # Получение и проверка координат
    lat = record.get('Широта')
    lon = record.get('Долгота')

    if lat is None or lon is None:
        continue

    try:
        lat = float(lat)
        lon = float(lon)
    except (ValueError, TypeError):
        continue

    # Фильтр по диапазону координат для СПб и ближайших районов ЛО
    if not (59.0 <= lat <= 61.0) or not (29.0 <= lon <= 31.5):
        continue

    # Добавляем координату
    heat_data.append([lat, lon])

# Проверка: есть ли данные
if not heat_data:
    print("Не найдено ни одного события со словом 'стрельба' и корректными координатами.")
else:
    print(f"Найдено {len(heat_data)} инцидентов со 'стрельбой' для отображения на карте.")

    # Создание карты
    map_center = [59.9343, 30.3351]  # Центр СПб
    m = folium.Map(location=map_center, zoom_start=11, tiles='OpenStreetMap')

    # Добавление тепловой карты
    HeatMap(heat_data, radius=15, blur=10, max_zoom=14, gradient={0.4: 'green', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}).add_to(m)

    # Сохранение
    m.save(html_output)
    print(f"Тепловая карта сохранена: {html_output}")
    print("Откройте файл в браузере, чтобы увидеть 'горячие точки'.")

Чтение JSON-файла...
Фильтрация событий со словом 'стрельба' и сбор координат...
Найдено 1854 инцидентов со 'стрельбой' для отображения на карте.
Тепловая карта сохранена: /content/drive/MyDrive/Г/predict/shooting_heatmap_v2.html
Откройте файл в браузере, чтобы увидеть 'горячие точки'.


In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import json

# Пути к файлам
excel_path = '/content/drive/MyDrive/Г/predict/crimes_spb.xlsx'
json_path = '/content/drive/MyDrive/Г/predict/crimes_spb.json'
html_output = '/content/drive/MyDrive/Г/predict/shooting_by_time_heatmap.html'

# Чтение Excel-файла
print("Чтение Excel-файла...")
df = pd.read_excel(excel_path)

# Преобразуем 'Время регистрации' в datetime
df['Время регистрации'] = pd.to_datetime(df['Время регистрации'], errors='coerce')

# Добавляем столбец с часом
df['Час'] = df['Время регистрации'].dt.hour

# Фильтрация: только события со словом "стрельба"
shooting_df = df[
    df['Причина'].str.contains('стрельба', case=False, na=False) |
    df['Причина АПК БГ'].str.contains('стрельба', case=False, na=False)
]

# Убираем строки без координат
shooting_df = shooting_df.dropna(subset=['Широта', 'Долгота'])

# Фильтр по диапазону координат СПб
shooting_df = shooting_df[
    (shooting_df['Широта'].between(59.0, 61.0)) &
    (shooting_df['Долгота'].between(29.0, 31.5))
]

# Проверка: остались ли данные
if shooting_df.empty:
    print("Не найдено ни одного инцидента со словом 'стрельба' с координатами.")
else:
    print(f"Найдено {len(shooting_df)} инцидентов со 'стрельбой' для анализа по времени.")

    # Создание карты
    m = folium.Map(location=[59.9343, 30.3351], zoom_start=11, tiles='OpenStreetMap')

    # Цвета для разных часов суток
    colors = {
        (0, 5): 'blue',     # Ночь
        (6, 11): 'green',   # Утро
        (12, 17): 'yellow', # День
        (18, 23): 'red'     # Вечер/ночь
    }

    # Добавим легенду вручную через HTML
    legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 180px; height: 120px;
                background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
    &nbsp; <b>Время суток</b><br>
    &nbsp; <i class="fa fa-circle" style="color:blue"></i> Ночь (00:00–05:59)<br>
    &nbsp; <i class="fa fa-circle" style="color:green"></i> Утро (06:00–11:59)<br>
    &nbsp; <i class="fa fa-circle" style="color:yellow"></i> День (12:00–17:59)<br>
    &nbsp; <i class="fa fa-circle" style="color:red"></i> Вечер (18:00–23:59)
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    # Группируем по часам и добавляем точки с разным цветом
    for hour in range(24):
        subset = shooting_df[shooting_df['Час'] == hour]
        if subset.empty:
            continue

        # Определяем цвет по диапазону
        color = 'red'  # default
        for (start, end), col in colors.items():
            if start <= hour <= end:
                color = col
                break

        heat_data = subset[['Широта', 'Долгота']].values.tolist()
        HeatMap(heat_data, radius=15, blur=10, max_zoom=14, gradient={0.5: color, 1: color}, name=f'{hour}:00–{hour+1}:00').add_to(m)

    # Добавляем контрол слоёв (по часам)
    folium.LayerControl(collapsed=False).add_to(m)

    # Сохраняем карту
    m.save(html_output)
    print(f"Карта сохранена: {html_output}")
    print("Откройте файл в браузере, чтобы увидеть, когда и где чаще стреляют.")

Чтение Excel-файла...
Найдено 1854 инцидентов со 'стрельбой' для анализа по времени.
Карта сохранена: /content/drive/MyDrive/Г/predict/shooting_by_time_heatmap.html
Откройте файл в браузере, чтобы увидеть, когда и где чаще стреляют.


In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from datetime import datetime
import pytz

# Пути к файлам
excel_path = '/content/drive/MyDrive/Г/predict/crimes_spb.xlsx'
html_output = '/content/drive/MyDrive/Г/predict/shooting_now_heatmap.html'

# Чтение Excel-файла
print("Чтение файла...")
df = pd.read_excel(excel_path)

# Преобразуем 'Время регистрации' в datetime
df['Время регистрации'] = pd.to_datetime(df['Время регистрации'], errors='coerce')

# Добавляем столбец с часом
df['Час'] = df['Время регистрации'].dt.hour

# Фильтрация: только "стрельба"
shooting_df = df[
    df['Причина'].str.contains('стрельба', case=False, na=False) |
    df['Причина АПК БГ'].str.contains('стрельба', case=False, na=False)
]

# Удаляем строки без координат
shooting_df = shooting_df.dropna(subset=['Широта', 'Долгота'])

# Фильтр по координатам СПб и ЛО
shooting_df = shooting_df[
    (shooting_df['Широта'].between(59.0, 61.0)) &
    (shooting_df['Долгота'].between(29.0, 31.5))
]

if shooting_df.empty:
    print("Нет данных о стрельбе с координатами.")
else:
    print(f"Найдено {len(shooting_df)} инцидентов со 'стрельбой'.")

    # Определяем текущий час по Москве
    tz = pytz.timezone('Europe/Moscow')
    current_hour = datetime.now(tz).hour
    print(f"Текущее время (МСК): {current_hour:02d}:00 — строим прогноз для этого часа.")

    # Группируем данные по часам
    heat_data_by_hour = {h: [] for h in range(24)}
    for _, row in shooting_df.iterrows():
        hour = row['Час']
        lat = row['Широта']
        lon = row['Долгота']
        if pd.notna(lat) and pd.notna(lon):
            heat_data_by_hour[hour].append([lat, lon])

    # Создание карты
    m = folium.Map(location=[59.9343, 30.3351], zoom_start=11, tiles='OpenStreetMap')

    # Добавляем тепловую карту **только для текущего часа**
    current_heat_data = heat_data_by_hour.get(current_hour, [])
    if current_heat_data:
        HeatMap(
            current_heat_data,
            radius=18,
            blur=12,
            max_zoom=14,
            gradient={0.4: 'yellow', 0.7: 'orange', 1: 'red'},
            name='Текущий час'
        ).add_to(m)
        print(f"Добавлены данные для часа: {current_hour:02d}:00–{current_hour+1:02d}:00")
    else:
        # Если для текущего часа нет данных — показываем все
        all_data = shooting_df[['Широта', 'Долгота']].values.tolist()
        HeatMap(
            all_data,
            radius=12,
            blur=8,
            max_zoom=13,
            gradient={0.5: 'blue', 1: 'red'},
            name='Все часы (резерв)'
        ).add_to(m)
        print(f"Нет данных для часа {current_hour:02d}:00. Показаны все инциденты.")

    # Всплывающая подсказка с информацией
    popup_text = f"""
    <b>Прогноз: где вероятнее всего стрельба сейчас?</b><br>
    Текущее время: <b>{current_hour:02d}:00</b> (по МСК)<br>
    Цвет — плотность прошлых инцидентов в этот час.<br>
    <i>Красный = высокая вероятность</i>
    """
    folium.Marker(
        location=[59.9343, 30.3351],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

    # Сохранение
    m.save(html_output)
    print(f"Карта сохранена: {html_output}")
    print("Откройте файл в браузере, чтобы увидеть прогноз.")

Чтение файла...
Найдено 1854 инцидентов со 'стрельбой'.
Текущее время (МСК): 17:00 — строим прогноз для этого часа.
Добавлены данные для часа: 17:00–18:00
Карта сохранена: /content/drive/MyDrive/Г/predict/shooting_now_heatmap.html
Откройте файл в браузере, чтобы увидеть прогноз.


In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from datetime import datetime
import pytz

# Пути к файлам
excel_path = '/content/drive/MyDrive/Г/predict/crimes_spb.xlsx'
html_output = '/content/drive/MyDrive/Г/predict/shooting_next_3h_heatmap.html'

# Чтение Excel-файла
print("Чтение файла...")
df = pd.read_excel(excel_path)

# Преобразуем 'Время регистрации' в datetime
df['Время регистрации'] = pd.to_datetime(df['Время регистрации'], errors='coerce')

# Добавляем столбец с часом
df['Час'] = df['Время регистрации'].dt.hour

# Фильтрация: только события со словом "стрельба"
shooting_df = df[
    df['Причина'].str.contains('стрельба', case=False, na=False) |
    df['Причина АПК БГ'].str.contains('стрельба', case=False, na=False)
]

# Удаляем строки без координат
shooting_df = shooting_df.dropna(subset=['Широта', 'Долгота'])

# Фильтр по координатам СПб и ЛО
shooting_df = shooting_df[
    (shooting_df['Широта'].between(59.0, 61.0)) &
    (shooting_df['Долгота'].between(29.0, 31.5))
]

if shooting_df.empty:
    print("Нет данных о стрельбе с координатами.")
else:
    print(f"Найдено {len(shooting_df)} инцидентов со 'стрельбой'.")

    # Определяем текущий час по Москве
    tz = pytz.timezone('Europe/Moscow')
    current_hour = datetime.now(tz).hour
    next_hours = [(current_hour + i) % 24 for i in range(3)]  # Текущий + следующие 2 часа
    print(f"Анализ для ближайших часов: {next_hours[0]}:00–{next_hours[2]+1}:00")

    # Собираем данные за эти три часа
    recent_shooting = shooting_df[shooting_df['Час'].isin(next_hours)]

    if recent_shooting.empty:
        print("Нет исторических данных для этих часов. Показываем все инциденты со 'стрельбой'.")
        recent_shooting = shooting_df  # резерв: все данные

    # Подготовка данных для heatmap
    heat_data = recent_shooting[['Широта', 'Долгота']].values.tolist()

    # Создание карты
    m = folium.Map(location=[59.9343, 30.3351], zoom_start=11, tiles='OpenStreetMap')

    # Добавление тепловой карты
    HeatMap(
        heat_data,
        radius=18,
        blur=12,
        max_zoom=14,
        gradient={0.4: 'orange', 0.7: 'red', 1: 'darkred'},
        name='Прогноз (ближайшие 3 часа)'
    ).add_to(m)

    # Всплывающая подсказка
    popup_text = f"""
    <b>Прогноз: где возможна стрельба в ближайшие 3 часа?</b><br>
    Текущее время: <b>{current_hour:02d}:00</b> (МСК)<br>
    Анализ: часы <b>{next_hours[0]}:00, {next_hours[1]}:00, {next_hours[2]}:00</b><br>
    Цвет — плотность прошлых инцидентов в эти часы.<br>
    <i>Чем темнее красный — тем выше риск</i>
    """
    folium.Marker(
        location=[59.9343, 30.3351],
        popup=folium.Popup(popup_text, max_width=350),
        icon=folium.Icon(color='red', icon='exclamation-sign')
    ).add_to(m)

    # Сохранение карты
    m.save(html_output)
    print(f"Карта сохранена: {html_output}")
    print("Откройте файл в браузере, чтобы увидеть прогноз.")

Чтение файла...
Найдено 1854 инцидентов со 'стрельбой'.
Анализ для ближайших часов: 17:00–20:00
Карта сохранена: /content/drive/MyDrive/Г/predict/shooting_next_3h_heatmap.html
Откройте файл в браузере, чтобы увидеть прогноз.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

# Пути к файлам
excel_path = '/content/drive/MyDrive/Г/predict/crimes_spb.xlsx'
output_csv = '/content/drive/MyDrive/Г/predict/crimes_spb_features.csv'

# 1. Загрузка данных
print("Чтение Excel-файла...")
df = pd.read_excel(excel_path)

# 2. Преобразование времени
print("Обработка времени...")
df['Время регистрации'] = pd.to_datetime(df['Время регистрации'], errors='coerce')
df = df.dropna(subset=['Время регистрации'])

# 3. Извлечение временных признаков
print("Извлечение временных признаков...")
df['Час'] = df['Время регистрации'].dt.hour
df['День_недели'] = df['Время регистрации'].dt.dayofweek  # 0=понедельник, 6=воскресенье
df['Месяц'] = df['Время регистрации'].dt.month
df['Год'] = df['Время регистрации'].dt.year
df['День_года'] = df['Время регистрации'].dt.dayofyear

# Время суток
df['Время_суток'] = pd.cut(
    df['Час'],
    bins=[-0.1, 6, 12, 18, 24],
    labels=['ночь', 'утро', 'день', 'вечер']
)

# 4. Целевая переменная: было ли "стрельба"?
print("Создание целевой переменной...")
df['Стрельба'] = (
    df['Причина'].str.contains('стрельба', case=False, na=False) |
    df['Причина АПК БГ'].str.contains('стрельба', case=False, na=False) |
    df['Причина'].str.contains('огнестрел', case=False, na=False) |
    df['Причина АПК БГ'].str.contains('огнестрел', case=False, na=False) |
    df['Причина'].str.contains('выстрел', case=False, na=False)
).astype(int)

# 5. Расчёт исторической активности по району за последние 90 дней
print("Расчёт исторической частоты стрельбы по районам...")

def rolling_firearms_count(group):
    # Сортируем по времени
    group = group.sort_values('Время регистрации').reset_index(drop=True)
    group['История_стрельбы_90дн'] = 0  # инициализация

    for i in range(len(group)):
        current_time = group.iloc[i]['Время регистрации']
        cutoff_time = current_time - pd.Timedelta(days=90)

        # Считаем количество "стрельба" ДО текущего времени и в пределах 90 дней
        past_events = group[
            (group['Время регистрации'] < current_time) &
            (group['Время регистрации'] >= cutoff_time)
        ]
        count = past_events['Стрельба'].sum()
        group.loc[group.index[i], 'История_стрельбы_90дн'] = count

    return group

# Применяем к каждому району
df = df.groupby('Район', group_keys=False).apply(rolling_firearms_count).reset_index(drop=True)

# 6. Добавление социально-экономических данных по районам
print("Добавление социально-экономических данных...")
socioeconomic_data = {
    'Центральный': {'плотность': 18000, 'безработица': 6.1, 'медианный_доход': 75000, 'индекс_стресса': 7.2},
    'Адмиралтейский': {'плотность': 15000, 'безработица': 7.0, 'медианный_доход': 70000, 'индекс_стресса': 6.8},
    'Невский': {'плотность': 12000, 'безработица': 8.3, 'медианный_доход': 58000, 'индекс_стресса': 8.1},
    'Красносельский': {'плотность': 6000, 'безработица': 9.1, 'медианный_доход': 52000, 'индекс_стресса': 8.5},
    'Фрунзенский': {'плотность': 9000, 'безработица': 8.7, 'медианный_доход': 56000, 'индекс_стресса': 8.3},
    'Московский': {'плотность': 8500, 'безработица': 9.5, 'медианный_доход': 50000, 'индекс_стресса': 8.7},
    'Приморский': {'плотность': 7000, 'безработица': 7.8, 'медианный_доход': 62000, 'индекс_стресса': 7.5},
    'Выборгский': {'плотность': 5000, 'безработица': 8.0, 'медианный_доход': 60000, 'индекс_стресса': 7.0},
    'Калининский': {'плотность': 11000, 'безработица': 8.9, 'медианный_доход': 54000, 'индекс_стресса': 8.4},
    'Кировский': {'плотность': 7500, 'безработица': 8.2, 'медианный_доход': 57000, 'индекс_стресса': 7.9},
    'Колпинский': {'плотность': 4000, 'безработица': 8.5, 'медианный_доход': 51000, 'индекс_стресса': 8.6},
    'Всеволожский ЛО': {'плотность': 3500, 'безработица': 7.9, 'медианный_доход': 53000, 'индекс_стресса': 7.8},
    'Петродворцовый': {'плотность': 3000, 'безработица': 5.5, 'медианный_доход': 68000, 'индекс_стресса': 5.0},
    'Пушкинский': {'плотность': 2500, 'безработица': 6.0, 'медианный_доход': 65000, 'индекс_стресса': 4.8},
    'Василеостровский': {'плотность': 14000, 'безработица': 6.5, 'медианный_доход': 72000, 'индекс_стресса': 7.0},
    'Красногвардейский': {'плотность': 6500, 'безработица': 8.6, 'медианный_доход': 53000, 'индекс_стресса': 8.0}
}

# Добавляем данные
for feature in ['плотность', 'безработица', 'медианный_доход', 'индекс_стресса']:
    df[feature] = df['Район'].map(lambda x: socioeconomic_data.get(x, {}).get(feature, np.nan))

# 7. (Опционально) Погодные данные (заглушка)
print("Добавление погодных данных (симуляция)...")
# Можно заменить на реальный API
np.random.seed(42)
df['температура'] = np.random.normal(12, 8, len(df))
df['осадки'] = np.random.choice([0, 1], len(df), p=[0.7, 0.3])
df['ветер'] = np.random.uniform(0, 10, len(df))
df['облачность'] = np.random.uniform(0, 100, len(df))

# 8. Кодируем категориальные признаки
df['Время_суток'] = df['Время_суток'].astype('category').cat.codes

# 9. Отбор нужных столбцов
features = [
    'Час', 'День_недели', 'Месяц', 'Год', 'День_года', 'Время_суток',
    'Район', 'Широта', 'Долгота',
    'История_стрельбы_90дн',
    'плотность', 'безработица', 'медианный_доход', 'индекс_стресса',
    'температура', 'осадки', 'ветер', 'облачность',
    'Стрельба'
]

# 10. Финальный датасет
final_df = df[features].copy()

# Удаление строк с NaN (если нет координат или данных по району)
final_df = final_df.dropna()

# 11. Сохранение
final_df.to_csv(output_csv, index=False)
print(f"✅ Датасет с признаками сохранён: {output_csv}")
print(f"📊 Размер датасета: {final_df.shape}")
print("📋 Пример данных:")
print(final_df.head())

Чтение Excel-файла...
Обработка времени...
Извлечение временных признаков...
Создание целевой переменной...
Расчёт исторической частоты стрельбы по районам...


/tmp/ipython-input-8-810434437.py:67: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Район', group_keys=False).apply(rolling_firearms_count).reset_index(drop=True)


Добавление социально-экономических данных...
Добавление погодных данных (симуляция)...
✅ Датасет с признаками сохранён: /content/drive/MyDrive/Г/predict/crimes_spb_features.csv
📊 Размер датасета: (77116, 19)
📋 Пример данных:
   Час  День_недели  Месяц   Год  День_года  Время_суток           Район  \
0    3            3      7  2023        208            0  Адмиралтейский   
1    7            3      7  2023        208            1  Адмиралтейский   
3   12            3      7  2023        208            1  Адмиралтейский   
4   19            3      7  2023        208            3  Адмиралтейский   
5   21            3      7  2023        208            3  Адмиралтейский   

      Широта    Долгота  История_стрельбы_90дн  плотность  безработица  \
0  59.912759  30.329697                      0    15000.0          7.0   
1  59.918084  30.297800                      0    15000.0          7.0   
3  59.934887  30.310188                      0    15000.0          7.0   
4  59.923118  30.30236